### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_tsumego

CONCEPT_NAME = concepts.concept_tsumego.__name__


In [21]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [25]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.5 # The higher, the more unique games. 0 = argmax, 1 = distribution 

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 250 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   1%|          | 2/250 [00:02<05:33,  1.35s/it]
Positive cases: 251it [04:33,  1.23it/s]                         

In [26]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (250, 4, 5, 5)
Negative cases:  (250, 4, 5, 5)


In [27]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [28]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  1
Duplicate count in negative:  15


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [29]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 400


In [30]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
13/13 [==============================] - 0s 10ms/step - loss: 0.7952 - val_loss: 0.8290
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7916 - val_loss: 0.8260
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7881 - val_loss: 0.8231
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7850 - val_loss: 0.8203
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7820 - val_loss: 0.8177
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7794 - val_loss: 0.8154
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7768 - val_loss: 0.8134
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7748 - val_loss: 0.8116
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7729 - val_loss: 0.8100
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7711 - val_loss: 0.8086
Epoch 11/50
13/13 

In [31]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 16/16 [00:00<00:00, 284.81it/s]

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 0s 9ms/step - loss: 0.9820 - val_loss: 0.9970
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9758 - val_loss: 0.9919
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9699 - val_loss: 0.9873
Epoch 4/50
13/13 [==============================] - 0s 2ms/step - loss: 0.9639 - val_loss: 0.9823
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9583 - val_loss: 0.9776
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9526 - val_loss: 0.9725
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9471 - val_loss: 0.9678
Epoch 8/50
13/13 [==============================] - 0s 2ms/step - loss: 0.9417 - val_loss: 0.9628
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9365 - val_loss: 0.9582
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9311 - val_loss: 0.9534
Epoch 11/50
13/13 [===========

Getting resblock outputs: 100%|██████████| 16/16 [00:00<00:00, 252.01it/s]

Performing regression for layer 0
Epoch 1/50


...

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 0s 10ms/step - loss: 1.0223 - val_loss: 0.9704
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 1.0105 - val_loss: 0.9598
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9993 - val_loss: 0.9502
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9889 - val_loss: 0.9405
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9787 - val_loss: 0.9316
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9687 - val_loss: 0.9230
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9594 - val_loss: 0.9141
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9503 - val_loss: 0.9058
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9411 - val_loss: 0.8972
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9320 - val_loss: 0.8893
Epoch 11/50
13/13 [==========

Getting resblock outputs: 100%|██████████| 16/16 [00:00<00:00, 274.92it/s]

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 0s 12ms/step - loss: 0.9348 - val_loss: 0.9044
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9189 - val_loss: 0.8903
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9065 - val_loss: 0.8776
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8956 - val_loss: 0.8660
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8852 - val_loss: 0.8565
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8760 - val_loss: 0.8467
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8675 - val_loss: 0.8369
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8592 - val_loss: 0.8281
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8516 - val_loss: 0.8202
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8443 - val_loss: 0.8130
Epoch 11/50
13/13 [==========